# Interest Rate Indices

This notebook demonstrates how to access and use the functionalities of index templates which are part of our 'lseg_analytics.pricing.reference_data.floating_rate_indices' module within LSEG Financial Analytics SDK.

## Imports

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

The following modules will be required in this notebook:

In [1]:
import pandas as pd
import json
from lseg_analytics.pricing.reference_data.floating_rate_indices import search, load
from lseg_analytics.pricing.common import SortingOrderEnum
from IPython.display import display

## Utility methods

In [2]:
def extract_tag_key(tag):
    return tag.split(":")[0] if ":" in tag else tag

def list_unique_tags(all_swaps):
    unique_tags = set()
    for item in all_swaps:
        tags = item.get("description", {}).get("tags", [])
        for tag in tags:
            key = extract_tag_key(tag)
            unique_tags.add(key)
    return unique_tags

def display_templates(templates):
    unique_tag_keys = list(list_unique_tags(templates))

    rows = []
    for item in templates:
        row = {
            "Space": item.get("location", {}).get("space", ""),
            "Id": item.get("id", ""),
            "Name": item.get("location", {}).get("name", ""),
            "Summary": item.get("description", {}).get("summary", ""),
        }
        tags = item.get("description", {}).get("tags", [])
        tag_dict = {extract_tag_key(tag): tag for tag in tags}
        for key in unique_tag_keys:
            tag_val = tag_dict.get(key, None)
            if tag_val is not None and ":" in tag_val:
                row[key] = tag_val.split(":", 1)[1]
            else:
                row[key] = tag_val
        rows.append(row)

    display(pd.DataFrame(rows))

## Searching for LSEG index templates

Retrieve 3 index templates from the 'LSEG' space, sorted by name in ascending order.

In [3]:
index_templates = search(item_per_page= 3, spaces=["LSEG"], space_name_sort_order = SortingOrderEnum.ASC)

display_templates(index_templates)

,Space,Id,Name,Summary,sourceLongName,indexTenor,sourceShortName,currency
0,LSEG,f882ded9-13b9-4456-ab56-134d93cec3d4,AED_AEIBOR_1M,AED AEIBOR,Refinitiv,1M,RFTB,AED
1,LSEG,7a102320-d152-4ebf-9fce-b4b254ddc471,AED_AEIBOR_1Y,AED AEIBOR,Refinitiv,1Y,RFTB,AED
2,LSEG,3d8b26aa-940a-45d0-9756-6af902ce5738,AED_AEIBOR_3M,AED AEIBOR,Refinitiv,3M,RFTB,AED


Retrieve overnight (indexTenor = ON) index templates in US Dollar (USD), contributed by RFTB, from the LSEG space.

In [4]:
usd_templates = search(tags=["currency:USD", "indexTenor:ON"], spaces=["LSEG"])

display_templates(usd_templates)

,Space,Id,Name,Summary,sourceLongName,indexTenor,sourceShortName,currency
0,LSEG,710fc04c-e793-44f4-a0f1-b133c8a37c87,USD_FFER_ON,Fed Funds Effective Rate,Refinitiv,ON,RFTB,USD
1,LSEG,3f3599e5-2f88-4c09-a1e0-aa3d9529e647,USD_LIBOR_ON_IBA,USD LIBOR (Ice Benchmark Administration),Ice Benchmark Administration,ON,IBA,USD
2,LSEG,5d70b006-035c-4862-8585-8bbecbbeb81a,USD_SOFR_ON_FRBNY,US Dollar Secured Overnight Financial Rate SOF...,FEDERAL RESERVE BANK OF NY,ON,FRBNY,USD
3,LSEG,78c319ae-cf6f-43f0-9ca4-1c4e9ef6702e,USD_USFFTARGET_ON,Fed Funds Target Rate,Refinitiv,ON,RFTB,USD


## Exploring features of a specific index template

Let's explore the specificities of the SOFR swap template, via Space/Name as 'resource_id'

In [5]:
sofr_name = [item["location"]["name"] for item in usd_templates if "SOFR" in item["location"]["name"]][0]

print(sofr_name)

sofr_index = load(name=sofr_name)

print(json.dumps(sofr_index.definition.as_dict(), indent=4))

USD_SOFR_ON_FRBNY
{
    "currency": "USD",
    "name": "SOFR",
    "tenor": "ON",
    "yearBasis": 0,
    "rounding": {
        "decimalPlaces": 0,
        "scale": 1
    },
    "quoteDefinition": {
        "instrumentCode": "USDSOFR=",
        "source": "FRBNY"
    }
}


Now let's explore again this swap template, this time via its unique Id.

In [6]:
sofr_id = usd_templates[0].get("id", None)
print(sofr_id)

sofr_index = load(resource_id=sofr_id)

print(json.dumps(sofr_index.definition.as_dict(), indent=4))

710fc04c-e793-44f4-a0f1-b133c8a37c87
{
    "currency": "USD",
    "name": "FFER",
    "tenor": "ON",
    "yearBasis": 0,
    "rounding": {
        "decimalPlaces": 0,
        "scale": 1
    },
    "quoteDefinition": {
        "instrumentCode": "USONFFE=",
        "source": "RFTB"
    }
}


And finally let's see how we can get a description of the same index template, this time via Space/Name.

In [7]:
sofr_index = load(resource_id="LSEG/" + sofr_name)

print(json.dumps(sofr_index.definition.as_dict(), indent=4))

{
    "currency": "USD",
    "name": "SOFR",
    "tenor": "ON",
    "yearBasis": 0,
    "rounding": {
        "decimalPlaces": 0,
        "scale": 1
    },
    "quoteDefinition": {
        "instrumentCode": "USDSOFR=",
        "source": "FRBNY"
    }
}
